### Задание

1. Обучить модель на понравившихся данных
2. Создать rest api сервис, к которому можно будет обращаться для получения прогнозов

### Примерные темы

1. Определение токсичности комментария
2. Определение стоимости жилья от разных факторов: площадь жилья, удаленность от центра, etc (здесь же куча идей прогнозирования стоимости чего-либо по его описанию)
3. Темы новости по ее тексту
4. Классификатор рукописных цифр
5. Классификатор картинок (например, определение города и страны по снимку улицы)
6. Определение вероятности наличия сердечно-сосудистых заболеваний по данным первичного осмотра (или что-то подобное)
7. Многое другое (придумайте сами)
8. Прогнозирование рейтинга вопроса на stackoverflow - https://www.kaggle.com/imoore/60k-stack-overflow-questions-with-quality-rate
9. https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016
10. Тематическое моделирование статей на arxiv - https://www.kaggle.com/Cornell-University/arxiv

### Откуда брать данные

1. kaggle (https://www.kaggle.com/datasets)
2. спарсить самостоятельно, но в этом случае вам может понадобиться разметка (если у вас обучение с учителем)

Пишем простой граббер для Telegram чатов на Python https://medium.com/swlh/build-a-telegram-bot-in-go-in-9-minutes-e06ad38acef1  https://proglib.io/p/pishem-prostoy-grabber-dlya-telegram-chatov-na-python-2019-11-06

In [1]:
import pandas as pd
import numpy as np

In [5]:
ham_raw = pd.read_json('../../data/spam_detection/ham2016.json')
ham_raw.head(3)

,name,type,id,messages
0,Новое Измайлово,public_supergroup,9666949336,"{'id': -999996617, 'type': 'service', 'date': ..."
1,Новое Измайлово,public_supergroup,9666949336,"{'id': 1, 'type': 'service', 'date': '2016-08-..."
2,Новое Измайлово,public_supergroup,9666949336,"{'id': 99, 'type': 'service', 'date': '2016-09..."


In [7]:
ham = pd.json_normalize(ham_raw.iloc[:, 3])
pd.set_option('display.max_columns', 50)
ham.tail(3)

,id,type,date,actor,actor_id,action,text,title,members,from,from_id,file,thumbnail,media_type,sticker_emoji,width,height,reply_to_message_id,photo,edited,forwarded_from,mime_type,contact_information.first_name,contact_information.last_name,contact_information.phone_number,duration_seconds,via_bot,contact_vcard,inviter,message_id,poll.question,poll.closed,poll.total_voters,poll.answers,location_information.latitude,location_information.longitude
60987,73759,message,2021-03-24T09:52:51,NaN,NaN,NaN,мойка проходит в несколько этапов: сначала сби...,NaN,NaN,Серёжа,4.801717e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60988,73764,message,2021-03-24T10:08:35,NaN,NaN,NaN,Спасибо большое! Попробую сегодня,NaN,NaN,ИриШок,5.225578e+09,NaN,NaN,NaN,NaN,NaN,NaN,73759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60989,73765,message,2021-03-24T10:08:35,NaN,NaN,NaN,"[{'type': 'mention_name', 'text': 'Серёжа', 'u...",NaN,NaN,ChatKeeperBot,4.848115e+09,NaN,NaN,NaN,NaN,NaN,NaN,73759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Пример предпоследнего сообщения

In [8]:
ham['text'][60988]

'Спасибо большое! Попробую сегодня'

Пример последнего сообщения от бота

In [9]:
ham['text'][60989]

[{'type': 'mention_name', 'text': 'Серёжа', 'user_id': 506749612},
 ', вашу репутацию увеличил ',
 {'type': 'mention_name', 'text': 'ИриШок', 'user_id': 930610950},
 '.\n Ваша репутация 8']

Список признаков

In [10]:
ham.columns

Index(['id', 'type', 'date', 'actor', 'actor_id', 'action', 'text', 'title',
       'members', 'from', 'from_id', 'file', 'thumbnail', 'media_type',
       'sticker_emoji', 'width', 'height', 'reply_to_message_id', 'photo',
       'edited', 'forwarded_from', 'mime_type',
       'contact_information.first_name', 'contact_information.last_name',
       'contact_information.phone_number', 'duration_seconds', 'via_bot',
       'contact_vcard', 'inviter', 'message_id', 'poll.question',
       'poll.closed', 'poll.total_voters', 'poll.answers',
       'location_information.latitude', 'location_information.longitude'],
      dtype='object')

Уникальные значения по некоторым признакам

In [11]:
all_cols = ['id', 'type', 'date', 'actor', 'actor_id', 'action', 'text', 'title',
       'members', 'from', 'from_id', 'file', 'thumbnail', 'media_type',
       'sticker_emoji', 'width', 'height', 'reply_to_message_id', 'photo',
       'edited', 'forwarded_from', 'mime_type',
       'contact_information.first_name', 'contact_information.last_name',
       'contact_information.phone_number', 'duration_seconds', 'via_bot',
       'contact_vcard', 'inviter', 'message_id', 'poll.question',
       'poll.closed', 'poll.total_voters', 'poll.answers',
       'location_information.latitude', 'location_information.longitude']

some_cols = ['id', 'type', 'date', 'actor_id', 'action',  
         'from_id', 'file', 'thumbnail', 'media_type',
       'sticker_emoji', 'reply_to_message_id', 'photo',
       'edited',  'mime_type',
       'contact_information.first_name', 'contact_information.last_name',
       'contact_information.phone_number', 'duration_seconds', 'via_bot',
        'inviter', 'message_id', 'poll.question',
       'poll.closed', 'poll.total_voters', 
       'location_information.latitude', 'location_information.longitude']

for col in some_cols:
    print(col, ham[col].unique())

id [-999996617          1         99 ...      73759      73764      73765]
type ['service' 'message']
date ['2016-08-26T12:01:01' '2016-09-01T18:04:20' '2016-09-01T18:07:07' ...
 '2021-03-24T09:48:16' '2021-03-24T09:52:51' '2021-03-24T10:08:35']
actor_id [4.49716059e+09 9.66694934e+09 4.48584734e+09 ... 4.86776919e+09
 5.43563062e+09 5.47227313e+09]
action ['migrate_to_supergroup' 'migrate_from_group' 'invite_members' nan
 'remove_members' 'join_group_by_link' 'pin_message' 'group_call'
 'invite_to_group_call']
from_id [           nan 4.47956145e+09 4.55065251e+09 ... 5.37073416e+09
 5.95687004e+09 5.75695058e+09]
file [nan '(File not included. Change data exporting settings to download.)']
thumbnail [nan '(File not included. Change data exporting settings to download.)']
media_type [nan 'sticker' 'video_file' 'animation' 'voice_message' 'video_message'
 'audio_file']
sticker_emoji [nan '😠' '💩' '🎉' '🌚' '😊' '🙈' '😒' '🎮' '😂' '🙄' '👈' '😳' '🤔' '😌' '😞' '😋' '😍'
 '☺️' '😁' '👍' '✌' '💐' '😡' '🔥' '😐